In [1]:
import os
from dotenv import dotenv_values

In [2]:
config = dotenv_values("./keys/.env")

In [3]:
os.environ['FAISS_ENABLE_GPU']='ON' 


In [4]:
import os, tempfile
from langchain.prompts import PromptTemplate


from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain, RetrievalQA

from langchain_text_splitters import CharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from google.oauth2 import service_account
from dotenv import dotenv_values
import json
import vertexai
import uuid
import faiss   
import itertools
import time
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [5]:
config = dotenv_values("./keys/.env")
with open("./keys/complete-tube-421007-208a4862c992.json") as source:
    info = json.load(source)

vertex_credentials = service_account.Credentials.from_service_account_info(info)
vertexai.init(
    project=config["PROJECT"],
    location=config["REGION"],
    credentials=vertex_credentials,
)
google_api_key = config["GEMINI-API-KEY"]
os.environ["GEMINI_API_KEY"] = google_api_key

In [6]:
ROOT= os.getcwd()
ROOT

'D:\\repos2\\bot'

In [7]:
embeddings_retriever = GoogleGenerativeAIEmbeddings(
                        model="models/embedding-001",
                        credentials=vertex_credentials,
                        google_api_key=google_api_key,
                    )

In [8]:
embeddings_retriever

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002B517058FD0>, model='models/embedding-001', task_type=None, google_api_key=SecretStr('**********'), credentials=<google.oauth2.service_account.Credentials object at 0x000002B56D947450>, client_options=None, transport=None, request_options=None)

In [9]:
mypath = "./docs"
onlyfiles = [f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]
onlyfiles

['Parser Source 2.pdf']

In [10]:
def load_file(path):
    # load pdf file and transform into Langchain Documents
    loader = PyPDFLoader(path)
    pages = loader.load_and_split()
    return pages

def get_docs_to_add_vectorstore(pages, file, category="legal"):
    # get components to add to Faiis
    documents = []
    ids= []
    metadatas= []
   
    for page in pages:

        metadatas.append({"page": page.metadata.get("page"), "filename":file, "category":category})
        ids.append(uuid.uuid1().hex)
        documents.append(page.page_content)

        
    return  documents,  ids, metadatas 

In [15]:
# path = os.path.join("docs", onlyfiles[0])
# pages = load_file(path)

In [41]:
# add documents to Faiss
documents_all =[]
metadatas_all = []
ids_all = []
category= "legal"
for file in onlyfiles:
    path = os.path.join("docs", file)
    pages = load_file(path)
    print(f" Loaded file {file} with {len(pages)} ")
    documents,  ids, metadatas = get_docs_to_add_vectorstore(pages, file, category)
    # add documents to collection
    documents_all = documents_all + documents
    metadatas_all = metadatas_all + metadatas
    ids_all = ids_all + ids

    print(f"number of documents: {len(documents_all)}")

 Loaded file Parser Source 2.pdf with 196 
number of documents: 196


In [32]:
documents_all[1]

"Aardvark Constructions Limited  \nDate  Company Profile  Company Number  \n09/08/2024  123456  \n \nDate 09/08/2024  AARDVCONST/MCPARTLA -C Page 1 Main Details  \n Name:  Aardvark Constructions Limited  \n QuickRef:  AARDVCONST  \n Country:  United Kingdom  \n Company Number:  123456  \n Incorporated:  20/10/2020  \n Company Type:  Limited by Shares  \n Company Status:  Active  \n Dissolved:   \n Registered Email Address:   \n \nPrimary Addresses  \nRegistered Office:  6 Chancery  Road, London, WC2A 5DP, United Kingdom  \n \nAccounting Dates  \n Last Period End  Current Period End  Last Extended  \nAccounting Dates:  16/11/2022  16/11/2024  16/11/2022  \n \nCompliance Details  \n Last Signed  Filed  Next Overdue  Last Company \nData Request  \nConfirmation Statement:  17/02/2023    17/02/2023  03/03/2023    \n \n Last Signed  Filed  Revenue  Next Overdue  \nAccounts:  16/11/2022  16/11/2022  16/11/2022  16/11/2022  16/11/2024  16/08/2025  \nApproved by Board:  16/11/2022  16/11/2022  

In [13]:
db = FAISS.from_texts(texts=documents_all, embedding=embeddings_retriever, metadatas=metadatas_all, ids=ids_all)

In [14]:
print(db.index.ntotal)

196


In [15]:
db.save_local("./faiss", index_name="companies")

In [16]:
db_load_2 =FAISS.load_local(folder_path="./faiss/", embeddings=embeddings_retriever, index_name="companies", allow_dangerous_deserialization=True)

In [17]:
print(db_load_2.index.ntotal)

196


In [18]:
query = "Main Details"

In [19]:
docs2 = db_load_2.similarity_search_with_relevance_scores(query)

In [20]:
len(docs2)

4

In [21]:
docs2[0]

(Document(metadata={'page': 89, 'filename': 'Parser Source 2.pdf', 'category': 'legal'}, page_content='Signatories:  ................................ ......  \nNotes:  ................................ ...............   \nSigned On  \nInternal Reference Number:  .............  0008  \nContract Description:  ........................   \nTotal Consideration:  .........................  1,000,000.00  \nCurrency:  ................................ .........  EU-EUR (Euro)  \nOther Party Reference Number (if any):   \nMain Contact:  ................................ ...'),
 0.5117046520039444)

In [22]:
retriever = db.as_retriever(
                        search_kwargs={"k": 5},
     filter={"filename" :"Parser Source 2.pdf"}
                    )

In [23]:
llm = ChatGoogleGenerativeAI(
                    model="gemini-1.5-pro-001", credentials=vertex_credentials
                )

In [42]:
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
# filter={"source" :"docs\\Baremo 2015.pdf"}
chain = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, return_source_documents=True)

In [54]:
chat_history = []

query = """Provide Main Details of the company Aardvark Constructions Limited. Including following details:
Name:
Country:
Company Number:
Incorporated:
Company Type:
Company Status:
Primary Addresses Registered Office:
Accounting Dates:
Confirmation Statement:
"""
result = chain.invoke({"question": query, "chat_history": chat_history})

print(result['answer'])

Name: Aardvark Constructions Limited
Country: United Kingdom
Company Number: 123456
Incorporated: 20/10/2020
Company Type: Limited by Shares
Company Status: Active
Primary Addresses Registered Office: 6 Chancery Road, London, WC2A 5DP, United Kingdom
Accounting Dates: 16/11/2022 - 16/11/2024
Confirmation Statement: Last Signed - 17/02/2023; Filed - 17/02/2023; Next Overdue - 03/03/2023 



In [55]:
#result['source_documents']

In [56]:
chat_history.append((query, result["answer"]))

In [57]:
query2 = """From Management Details extract:
Managed By:
Managed By Email:
"""
result = chain.invoke({"question": query2, "chat_history": chat_history})
print(result['answer'])

The document contains the name of one individual listed as "Managed By" for Aardvark Constructions Limited: Caroline McPartland. Her email address is cmcpartland@diligent.com. 



In [58]:
chat_history.append((query2, result["answer"]))

In [59]:
query3 = """Past Names of the Company with their period """
result = chain.invoke({"question": query3 ,"chat_history": chat_history})
print(result['answer'])

Aardvark Constructions Limited had two previous names:

* **Aardvark Construction** from **20/10/2020** to **20/10/2021**. 
* **Aardvark and Son Ltd** from **20/10/2021** to **20/10/2022**. 



In [60]:
chat_history.append((query3, result["answer"]))

In [61]:
query4 = """Appointments Board Positions"""
result = chain.invoke({"question": query4 ,"chat_history": chat_history})
print(result['answer'])

## Appointments and Board Positions within Aardvark Constructions Limited:

**Board Positions:**

* **Abbles, James** (ABBLES -J): Director, Appointed: 19/04/2023, Job Title: Trainer
* **Abdreatta, Leopoldo** (ABDREATT -L): Director, Appointed: 18/10/2023, Job Title: Secretary
* **Adam, Nicole** (ADAMS -N): 
    * Alternate Director, Appointed: 04/04/2023, Job Title: CFO
    * Non-Executive Director, Appointed: 10/04/2024, Job Title: CFO
* **Alberts, Stoffel** (ALBERTS -S): Company Secretary, Appointed: 16/12/2022, Job Title: Accountant
* **Rutter, Gus** (RUTTER -G): Director, Appointed: 07/03/2024, Job Title: Director

**Past Appointments:**

* **Malek, Mohammed** (MALEK -M): Director, Appointed: 22/07/2022, Resigned: 18/01/2024 

